In [1]:
import sys
from utils import *
from instructions import Instruction
from settings import *
from uncompute import *

n = 1
input_file = "test_files/ult.btor2"
current_settings = get_btor2_settings(input_file)
Instruction.all_instructions = read_file(input_file, modify_memory_sort=False, setting=current_settings)
Instruction.with_grover = 0

for i in range(1, n+1):
    print(i)
    Instruction.current_n = i
    for instruction in Instruction.all_instructions.values():
        if instruction[1] == INIT and i == 1:
            Instruction(instruction).execute()
        elif instruction[1] == NEXT or instruction[1] == BAD:
            Instruction(instruction).execute()

result_bad_states = Instruction.or_bad_states()

print("stack: ", Instruction.global_stack.size)


1
stack:  127


In [2]:
Instruction.input_nids

[{1: (QuantumRegister(8, 'q2'), [-1, -1, -1, -1, -1, -1, -1, -1])},
 {1: (QuantumRegister(8, 'q3'), [-1, -1, -1, -1, -1, -1, -1, -1])}]

In [3]:
circuit_queue = get_circuit_queue(Instruction.global_stack)

In [4]:
def are_all_controls_true(values, controls):
    for c in controls:
        if values[c] == 0:
            return False
        else:
            assert(values[c] == 1)
    return True

def check_input(values):
    # we only set the value of the first input the other ones are set to |0>
    qubit_values = {}
    for (index,qword) in enumerate(Instruction.input_nids):
        for i in range(n+1):
            if i in qword.states.keys():
                temp_value = values[index]
                for qubit in qword.states[i][0]:
                    qubit_values[qubit] = temp_value % 2
                    temp_value = temp_value // 2
    element: Element = circuit_queue.pop()
    assert(element.element_type != CHECKPOINT_TYPE)
    while element.element_type != CHECKPOINT_TYPE:
        for o in element.operands:
            if o not in qubit_values.keys():
                qubit_values[o] = 0

        assert (element.target is not None)

        if element.target not in qubit_values.keys():
            qubit_values[element.target] = 0

        flip_target = True
        if element.gate_name == X:
            assert(len(element.operands) == 0)

        else:
            assert((element.gate_name == CX and len(element.operands) ==1) or
                   (element.gate_name == CCX and len(element.operands) == 2) or
                    element.gate_name == MCX)

            flip_target = are_all_controls_true(qubit_values, element.operands)
        if flip_target:
            qubit_values[element.target] = (qubit_values[element.target] + 1) % 2
        circuit_queue.push(element)
        element = circuit_queue.pop()
    assert element.element_type == CHECKPOINT_TYPE
    circuit_queue.push(element)
    result = 0
    for (index, qubit) in enumerate(Instruction.created_states_ids[4][1][0]):
        result += (2**index)*qubit_values[qubit]
    return result, qubit_values

In [6]:
for i in range(256):
    for j in range(256):
        result, values = check_input([i,j])
        local_result = i<j
        assert(local_result == values[Instruction.created_states_ids[5][1][0][0]])
        assert(result == local_result)